## Eclipse AM Project

In [ ]:
import pandas as pd
import os

## Create table of receiver stations
           if(iamd(jwl).eq.5)then
               rlat1=40.11
               rlong1=-88.21
            endif
            if(iamd(jwl).eq.6)then
               rlat1=42.44
               rlong1=-76.5
            endif
            if(iamd(jwl).eq.9)then
               rlat1=43.7
               rlong1=-72.29
            endif
            if(iamd(jwl).eq.11)then
               rlat1=42.33
               rlong1=-83.04
            endif
            if(iamd(jwl).eq.12)then
               rlat1=45.59
               rlong1=-92.97
            endif

## Read in data files and put into single dataframe:
50 files (one for each path) with XMTR lat, XMTR lon, RCVR Name encoded in the file name, and each file of format  Time, Signal Strength

I'm imagining a table that has a format along the lines of:
Time, Path, XMTR lat, XMTR lon, RCVR lat, RCVR lon, Signal Strength

In [ ]:
pd.read_csv("data/38.91-77.04_05", sep='\s+', header=None, names = ["Time", "Signal Strength"])

In [ ]:
foo = os.listdir("data/")[2]

foo[5:11]

In [ ]:
foo

In [ ]:
all_dfs = [] # preallocate
# Iterate over all files in the specified directory
for filename in os.listdir("data/"):
    filepath = os.path.join("data/", filename)

    # Ensure it's a file and not a directory
    if os.path.isfile(filepath):
        try:
            # Read each file into a pandas DataFrame
            # Using sep='\s+' to handle one or more whitespace characters as separators
            # header=None indicates no header row
            # names assigns column names
            df = pd.read_csv(filepath, sep='\s+', header=None, names=['Time', 'Signal Strength'])
            df['xmtr_lat'] = filename[0:5]
            df['xmtr_lon'] = filename[5:11]
            df["rcvr_num"] = filename[12:14]
            print(f"Successfully read {filename}:")
            print(df.head()) # Print first few rows of each file's DataFrame
            all_dfs.append(df)
        except Exception as e:
            print(f"Error reading file {filename}: {e}")

# Concatenate all DataFrames into a single DataFrame
if all_dfs:
    combined_df = pd.concat(all_dfs, ignore_index=True)
    print("\nCombined DataFrame from all files:")
    print(combined_df)
else:
    print("No data files were found or successfully processed in the directory.")


In [ ]:
combined_df

In [ ]:
receiver_coords = {
    "05": (40.11, -88.21),
    "06": (42.44, -76.50),
    "09": (43.70, -72.29),
    "11": (42.33, -83.04),
    "12": (45.59, -92.97)
}

# Use the map method to create the 'rx_lat' and 'rx_lon' columns
# We use a lambda function to extract the latitude and longitude from the tuple
# For any 'rcvr_num' not found in the dictionary, it will assign NaN.
combined_df['rx_lat'] = combined_df['rcvr_num'].map(lambda x: receiver_coords.get(x, (None, None))[0])
combined_df['rx_lon'] = combined_df['rcvr_num'].map(lambda x: receiver_coords.get(x, (None, None))[1])

combined_df


In [ ]:
print(combined_df.head())

## Create animated map with Plotly Express

In [ ]:
# import pandas as pd
# import plotly.express as px
# import plotly.graph_objects as go
# import numpy as np
# from matplotlib.colors import LinearSegmentedColormap # This is a good way to get intermediate colors

# # Fallback in case matplotlib is not available in the environment,
# # or for a simpler direct interpolation without external libs.
# # For now, let's assume matplotlib is available for better color interpolation.
# try:
#     from matplotlib.colors import LinearSegmentedColormap
#     USE_MATPLOTLIB_COLORMAP = True
# except ImportError:
#     USE_MATPLOTLIB_COLORMAP = False
#     print("Matplotlib not found, using simplified color interpolation.")


# def create_animated_paths_plot(df_original: pd.DataFrame):
#     """
#     Generates an animated Plotly Express geo plot showing paths from
#     transmitters to receivers, animated over time, with stations as scatter points.

#     The lines between transmitter and receiver are colormapped by 'Signal Strength'.
#     The transmitter and receiver stations themselves are displayed as distinct
#     scatter points.

#     Args:
#         df_original: A pandas DataFrame containing 'Time', 'Signal Strength',
#                      'xmtr_lat', 'xmtr_lon', 'rx_lat', and 'rx_lon' columns.
#     """
#     # Ensure 'Time' is sorted for correct animation frame order.
#     df_original = df_original.sort_values(by='Time').reset_index(drop=True)

#     unique_times = df_original['Time'].unique()
#     min_signal = df_original['Signal Strength'].min()
#     max_signal = df_original['Signal Strength'].max()

#     # Create a continuous colormap from Plotly's Plasma sequence
#     plasma_colors = px.colors.sequential.Plasma
#     if USE_MATPLOTLIB_COLORMAP:
#         cmap = LinearSegmentedColormap.from_list("plasma_cmap", plasma_colors)
#         def get_color_from_value(value, min_val, max_val):
#             if min_val == max_val:
#                 norm_value = 0.5
#             else:
#                 norm_value = (value - min_val) / (max_val - min_val)
#             # cmap returns RGBA tuple (0-1), convert to 'rgb(R,G,B)' string
#             rgba = cmap(norm_value)
#             return f'rgb({int(rgba[0]*255)},{int(rgba[1]*255)},{int(rgba[2]*255)})'
#     else:
#         # Fallback for simplified interpolation if matplotlib is not available
#         def get_color_from_value(value, min_val, max_val):
#             if min_val == max_val:
#                 norm_value = 0.5
#             else:
#                 norm_value = (value - min_val) / (max_val - min_val)
#             # Basic interpolation across the list of colors
#             idx = int(norm_value * (len(plasma_colors) - 1))
#             return plasma_colors[idx]


#     # Initialize the main figure with a base geo layout
#     fig = go.Figure(
#         layout=go.Layout(
#             title_text="Animated Paths and Stations: Transmitter to Receiver Over Time",
#             # This coloraxis is mainly for the color bar legend, as lines are colored manually per trace.
#             coloraxis=dict(
#                 colorbar=dict(
#                     title="Signal Strength",
#                     thicknessmode="pixels", thickness=20,
#                     lenmode="pixels", len=300,
#                     yanchor="middle", y=0.5,
#                     xanchor="right", x=1.05
#                 ),
#                 cmin=min_signal,
#                 cmax=max_signal,
#                 colorscale=plasma_colors # Use the same Plasma scale for the color bar
#             )
#         )
#     )

#     frames = []

#     # Iterate through each unique time value to create frames for the animation
#     for k, time_val in enumerate(unique_times):
#         frame_data = []
#         df_frame = df_original[df_original['Time'] == time_val]

#         # Add transmitter scatter point for this frame
#         frame_data.append(
#             go.Scattergeo(
#                 lon=[df_frame['xmtr_lon'].iloc[0]], # Assuming one transmitter per time step
#                 lat=[df_frame['xmtr_lat'].iloc[0]],
#                 mode='markers',
#                 marker=dict(
#                     size=15,
#                     color='darkorange',
#                     symbol='diamond',
#                     line=dict(width=1, color='Black') # Add a black border for visibility
#                 ),
#                 name='Transmitter Station',
#                 hoverinfo='text',
#                 text=f'Transmitter (Time: {time_val:.4f})',
#                 showlegend=(k == 0) # Show legend only for the first frame's trace
#             )
#         )

#         # Add receiver scatter point for this frame
#         frame_data.append(
#             go.Scattergeo(
#                 lon=[df_frame['rx_lon'].iloc[0]], # Assuming one receiver per time step
#                 lat=[df_frame['rx_lat'].iloc[0]],
#                 mode='markers',
#                 marker=dict(
#                     size=15,
#                     color='deepskyblue',
#                     symbol='circle',
#                     line=dict(width=1, color='Black') # Add a black border for visibility
#                 ),
#                 name=f'Receiver Station', # Generic name for legend
#                 hoverinfo='text',
#                 text=f'Receiver {df_frame["rcvr_num"].iloc[0]} (Time: {time_val:.4f})',
#                 showlegend=(k == 0) # Show legend only for the first frame's trace
#             )
#         )

#         # Add the line path for this frame
#         # Each line segment is a separate trace to allow individual color mapping
#         for index, row in df_frame.iterrows():
#             signal_strength = row['Signal Strength']
#             line_color = get_color_from_value(signal_strength, min_signal, max_signal)

#             frame_data.append(
#                 go.Scattergeo(
#                     lon=[row['xmtr_lon'], row['rx_lon']],
#                     lat=[row['xmtr_lat'], row['rx_lat']],
#                     mode='lines',
#                     line=dict(
#                         width=2,
#                         color=line_color # Set color based on signal strength
#                     ),
#                     hoverinfo='text',
#                     text=f'Signal Strength: {signal_strength}',
#                     showlegend=False, # Do not show legend for individual lines
#                 )
#             )
#         frames.append(go.Frame(data=frame_data, name=str(time_val)))

#     fig.frames = frames

#     # Set initial traces (for the very first frame) to ensure the plot is visible before animation starts
#     if frames:
#         fig.add_traces(frames[0].data)

#     # Update the layout to include animation controls (play/pause buttons and a time slider).
#     fig.update_layout(
#         updatemenus=[{
#             'buttons': [
#                 {
#                     'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
#                     'label': 'Play',
#                     'method': 'animate'
#                 },
#                 {
#                     'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
#                     'label': 'Pause',
#                     'method': 'animate'
#                 }
#             ],
#             'direction': 'left',
#             'pad': {'r': 10, 't': 87},
#             'showactive': False,
#             'type': 'buttons',
#             'x': 0.1,
#             'xanchor': 'right',
#             'y': 0,
#             'yanchor': 'top'
#         }],
#         sliders=[{
#             'active': 0,
#             'steps': [
#                 {
#                     'args': [[f.name], {'frame': {'duration': 500, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
#                     'label': f'{float(f.name):.4f}',
#                     'method': 'animate'
#                 } for f in fig.frames
#             ],
#             'transition': {'duration': 300},
#             'x': 0.1,
#             'len': 0.9,
#             'xanchor': 'left',
#             'y': 0,
#             'yanchor': 'top',
#             'pad': {'t': 50}
#         }],
#         # Geographic settings
#         geo=dict(
#             scope='world', # Set initial scope to world
#             projection_type="natural earth",
#             showcoastlines=True, coastlinecolor="Black",
#             showland=True, landcolor="LightGrey",
#             showocean=True, oceancolor="LightBlue",
#             showcountries=True, countrycolor="DarkGrey",
#             # Automatically fit bounds to the data. This will zoom into the relevant area.
#             # fitbounds="locations"
#             # Fixed bounds
#             lataxis_range=[35, 45], #lat_min_bound, lat_max_bound
#             lonaxis_range=[-75, -95] #lon_min_bound, lon_max_bound
#         ),
#         # Add legend for station types
#         showlegend=True,
#         legend=dict(
#             orientation="h",
#             yanchor="bottom",
#             y=1.02, # Position above the plot
#             xanchor="right",
#             x=1 # Position to the right
#         )
#     )

#     fig.show()
#     fig.write_html('eclipse_paths.html')

# # --- Example Usage (based on your provided table structure) ---
# if __name__ == "__main__":
#     # Sample DataFrame simulating your input data
#     # data = {
#     #     'Time': [8.009320, 8.018641, 8.027962, 8.037283, 8.046603,
#     #              8.050000, 8.060000, 8.070000, 8.080000, 8.090000],
#     #     'Signal Strength': [1351, 1424, 1351, 1296, 1337,
#     #                         1500, 1600, 1450, 1550, 1380],
#     #     'xmtr_lat': [38.91, 38.91, 38.91, 38.91, 38.91,
#     #                  39.00, 39.00, 39.00, 39.00, 39.00],
#     #     'xmtr_lon': [-77.04, -77.04, -77.04, -77.04, -77.04,
#     #                  -78.00, -78.00, -78.00, -78.00, -78.00],
#     #     'rcvr_num': [5, 5, 5, 5, 5,
#     #                  6, 9, 11, 12, 5],
#     #     'rx_lat': [40.11, 40.11, 40.11, 40.11, 40.11,
#     #                42.44, 43.70, 42.33, 45.59, 40.11],
#     #     'rx_lon': [-88.21, -88.21, -88.21, -88.21, -88.21,
#     #                -76.50, -72.29, -83.04, -92.97, -88.21]
#     # }
#     # sample_df = pd.DataFrame(data)

#     print("Generating animated plot with lines and scatter points using go.Figure...")
#     create_animated_paths_plot(combined_df)
#     print("Plot generated. It should appear in your browser or output environment.")


In [ ]:
combined_df[['xmtr_lat']].min()

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from matplotlib.colors import LinearSegmentedColormap # This is a good way to get intermediate colors

# Fallback in case matplotlib is not available in the environment,
# or for a simpler direct interpolation without external libs.
# For now, let's assume matplotlib is available for better color interpolation.
try:
    from matplotlib.colors import LinearSegmentedColormap
    USE_MATPLOTLIB_COLORMAP = True
except ImportError:
    USE_MATPLOTLIB_COLORMAP = False
    print("Matplotlib not found, using simplified color interpolation.")


def create_animated_paths_plot(df_original: pd.DataFrame):
    """
    Generates an animated Plotly Express geo plot showing paths from
    transmitters to receivers, animated over time, with stations as scatter points.

    The lines between transmitter and receiver are colormapped by 'Signal Strength'.
    The transmitter and receiver stations themselves are displayed as distinct
    scatter points.

    Args:
        df_original: A pandas DataFrame containing 'Time', 'Signal Strength',
                     'xmtr_lat', 'xmtr_lon', 'rx_lat', and 'rx_lon' columns.
    """
    # Ensure 'Time' is sorted for correct animation frame order.
    df_original = df_original.sort_values(by='Time').reset_index(drop=True)

    unique_times = df_original['Time'].unique()
    min_signal = df_original['Signal Strength'].min()
    max_signal = df_original['Signal Strength'].max()

    # Determine overall min/max lat/lon for fixed map bounds
    all_lats = pd.concat([df_original['xmtr_lat'], df_original['rx_lat']])
    all_lons = pd.concat([df_original['xmtr_lon'], df_original['rx_lon']])

    # Add a small buffer to the min/max for better visualization
    lat_min_bound = all_lats.min() - 2
    lat_max_bound = all_lats.max() + 2
    lon_min_bound = all_lons.min() - 2
    lon_max_bound = all_lons.max() + 2


    # Create a continuous colormap from Plotly's Plasma sequence
    plasma_colors = px.colors.sequential.Plasma

    if USE_MATPLOTLIB_COLORMAP:
        cmap = LinearSegmentedColormap.from_list("plasma_cmap", plasma_colors)
        def get_color_from_value(value, min_val, max_val):
            if min_val == max_val: # Handle case where min and max are the same
                norm_value = 0.5
            else:
                norm_value = (value - min_val) / (max_val - min_val)
            # Ensure norm_value is within [0, 1] range
            norm_value = max(0, min(1, norm_value))
            # cmap returns RGBA tuple (0-1), convert to 'rgb(R,G,B)' string
            rgba = cmap(norm_value)
            return f'rgb({int(rgba[0]*255)},{int(rgba[1]*255)},{int(rgba[2]*255)})'
    else:
        # Fallback for simplified interpolation if matplotlib is not available
        def get_color_from_value(value, min_val, max_val):
            if min_val == max_val:
                norm_value = 0.5
            else:
                norm_value = (value - min_val) / (max_val - min_val)
            norm_value = max(0, min(1, norm_value))
            idx = int(norm_value * (len(plasma_colors) - 1))
            return plasma_colors[idx]


    # Initialize the main figure with a base geo layout
    fig = go.Figure(
        layout=go.Layout(
            title_text="Animated Paths and Stations: Transmitter to Receiver Over Time",
            # This coloraxis configuration is now primarily for the color bar.
            # Lines will be explicitly colored with RGB strings.
            coloraxis=dict(
                colorbar=dict(
                    title="Signal Strength",
                    thicknessmode="pixels", thickness=20,
                    lenmode="pixels", len=300,
                    yanchor="middle", y=0.5,
                    xanchor="right", x=1.05
                ),
                cmin=min_signal,
                cmax=max_signal,
                colorscale=plasma_colors # Use the same Plasma scale for the color bar
            )
        )
    )

    # Add an invisible trace purely for the colorbar definition.
    # This trace carries the color scale information that the layout's coloraxis refers to.
    fig.add_trace(
        go.Scattergeo(
            lon=[df_original['xmtr_lon'].iloc[0]], # Use a valid coordinate from your data
            lat=[df_original['xmtr_lat'].iloc[0]], # Use a valid coordinate from your data
            mode='markers',
            marker=dict(
                # Crucially, pass a numerical array here to trigger the coloraxis mapping
                color=[min_signal, max_signal],
                colorscale=plasma_colors,
                cmin=min_signal,
                cmax=max_signal,
                showscale=True, # This trace is responsible for showing the colorbar
                size=1, # Make the marker very small
                opacity=0.001 # Make it almost completely transparent
            ),
            # Do NOT set visible=False, as this often hides the colorbar too.
            showlegend=False, # Don't show this dummy trace in the legend
            hoverinfo='none' # No hover info for this dummy trace
        )
    )

    frames = []

    # Iterate through each unique time value to create frames for the animation
    for k, time_val in enumerate(unique_times):
        frame_data = []
        df_frame = df_original[df_original['Time'] == time_val]

        # Add transmitter scatter point for this frame
        frame_data.append(
            go.Scattergeo(
                lon=[df_frame['xmtr_lon'].iloc[0]], # Assuming one transmitter per time step
                lat=[df_frame['xmtr_lat'].iloc[0]],
                mode='markers',
                marker=dict(
                    size=15,
                    color='darkorange',
                    symbol='diamond',
                    line=dict(width=1, color='Black') # Add a black border for visibility
                ),
                name='Transmitter Station',
                hoverinfo='text',
                text=f'Transmitter (Time: {time_val:.4f})',
                showlegend=(k == 0) # Show legend only for the first frame's trace
            )
        )

        # Add receiver scatter point for this frame
        frame_data.append(
            go.Scattergeo(
                lon=[df_frame['rx_lon'].iloc[0]], # Assuming one receiver per time step
                lat=[df_frame['rx_lat'].iloc[0]],
                mode='markers',
                marker=dict(
                    size=15,
                    color='deepskyblue',
                    symbol='circle',
                    line=dict(width=1, color='Black') # Add a black border for visibility
                ),
                name=f'Receiver Station', # Generic name for legend
                hoverinfo='text',
                text=f'Receiver {df_frame["rcvr_num"].iloc[0]} (Time: {time_val:.4f})',
                showlegend=(k == 0) # Show legend only for the first frame's trace
            )
        )

        # Add the line path for this frame
        # Line colors are now explicitly set using the get_color_from_value function.
        for index, row in df_frame.iterrows():
            signal_strength = row['Signal Strength']
            line_color_rgb_string = get_color_from_value(signal_strength, min_signal, max_signal)

            frame_data.append(
                go.Scattergeo(
                    lon=[row['xmtr_lon'], row['rx_lon']],
                    lat=[row['xmtr_lat'], row['rx_lat']],
                    mode='lines',
                    line=dict(
                        width=2,
                        color=line_color_rgb_string # Set color to the RGB string
                    ),
                    hoverinfo='text',
                    text=f'Signal Strength: {signal_strength}',
                    showlegend=False, # Do not show legend for individual lines
                )
            )
        frames.append(go.Frame(data=frame_data, name=str(time_val)))

    fig.frames = frames

    # Set initial traces (for the very first frame) to ensure the plot is visible before animation starts
    if frames:
        # Add the actual data for the first frame.
        # The dummy colorbar trace was already added above.
        fig.add_traces(frames[0].data)

    # Update the layout to include animation controls (play/pause buttons and a time slider).
    fig.update_layout(
        updatemenus=[{
            'buttons': [
                {
                    'args': [None, {'frame': {'duration': 500, 'redraw': True}, 'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                    'label': 'Play',
                    'method': 'animate'
                },
                {
                    'args': [[None], {'frame': {'duration': 0, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 0}}],
                    'label': 'Pause',
                    'method': 'animate'
                }
            ],
            'direction': 'left',
            'pad': {'r': 10, 't': 87},
            'showactive': False,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        }],
        sliders=[{
            'active': 0,
            'steps': [
                {
                    'args': [[f.name], {'frame': {'duration': 500, 'redraw': True}, 'mode': 'immediate', 'transition': {'duration': 300}}],
                    'label': f'{float(f.name):.4f}',
                    'method': 'animate'
                } for f in fig.frames
            ],
            'transition': {'duration': 300},
            'x': 0.1,
            'len': 0.9,
            'xanchor': 'left',
            'y': 0,
            'yanchor': 'top',
            'pad': {'t': 50}
        }],
        # Geographic settings
        geo=dict(
            scope='world', # Set initial scope to world
            projection_type="natural earth",
            showcoastlines=True, coastlinecolor="Black",
            showland=True, landcolor="LightGrey",
            showocean=True, oceancolor="LightBlue",
            showcountries=True, countrycolor="DarkGrey",
            # Fixed latitude and longitude ranges
            lataxis_range=[lat_min_bound, lat_max_bound],
            lonaxis_range=[lon_min_bound, lon_max_bound]
        ),
        # Add legend for station types
        showlegend=True,
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02, # Position above the plot
            xanchor="right",
            x=1 # Position to the right
        )
    )

    fig.show()
    fig.write_html('paths_with_colorbar.html')

# --- Example Usage (based on your provided table structure) ---
if __name__ == "__main__":
    # Sample DataFrame simulating your input data
    data = {
        'Time': [8.009320, 8.018641, 8.027962, 8.037283, 8.046603,
                 8.050000, 8.060000, 8.070000, 8.080000, 8.090000],
        'Signal Strength': [1351, 1424, 1351, 1296, 1337,
                            1500, 1600, 1450, 1550, 1380],
        'xmtr_lat': [38.91, 38.91, 38.91, 38.91, 38.91,
                     39.00, 39.00, 39.00, 39.00, 39.00],
        'xmtr_lon': [-77.04, -77.04, -77.04, -77.04, -77.04,
                     -78.00, -78.00, -78.00, -78.00, -78.00],
        'rcvr_num': [5, 5, 5, 5, 5,
                     6, 9, 11, 12, 5],
        'rx_lat': [40.11, 40.11, 40.11, 40.11, 40.11,
                   42.44, 43.70, 42.33, 45.59, 40.11],
        'rx_lon': [-88.21, -88.21, -88.21, -88.21, -88.21,
                   -76.50, -72.29, -83.04, -92.97, -88.21]
    }
    sample_df = pd.DataFrame(data)

    print("Generating animated plot with lines and scatter points using go.Figure...")
    combined_df['xmtr_lat'] = combined_df['xmtr_lat'].astype(float)
    combined_df['xmtr_lon'] = combined_df['xmtr_lon'].astype(float)
    combined_df['rx_lat'] = combined_df['rx_lat'].astype(float)
    combined_df['rx_lon'] = combined_df['rx_lon'].astype(float)
    create_animated_paths_plot(combined_df)
    print("Plot generated. It should appear in your browser or output environment.")
